In [ ]:
# Define possible chess figurines for validation and visualization
black_figurines = {
    "pawn": {"icon": "p", "limit": 8},
    "rook": {"icon": "r", "limit": 2},
    "knight": {"icon": "n", "limit": 2},
    "bishop": {"icon": "b", "limit": 2},
    "queen": {"icon": "q", "limit": 1},
    "king": {"icon": "k", "limit": 1}
}
white_figurines = {"bishop":"B", "pawn":"P"}

# Tracking used figurines
black_placements_count = {piece: 0 for piece in black_figurines}

# Function to check if a figurine can still be placed
def can_place_black_figurine(piece):
    return black_placements_count[piece] < black_figurines[piece]["limit"]

# Define an empty 8x8 board with position keys
board_positions = {f"{col}{row}": "" for col in "abcdefgh" for row in range(1, 9)}

# Function to print the board (visual representation)

def print_grid():
    print("     a     b     c     d     e     f     g     h  ")
    print("   ┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┐")

    for row in range(8, 0, -1):  # Reverse number order
        row_cells = []
        for col_idx, col in enumerate("abcdefgh"):
            cell = board_positions[f"{col}{row}"]

            # **Alternating colors**
            bg_color = "\033[47m" if (row + col_idx) % 2 == 0 else "\033[100m"

            # **Formatted cell with color**
            centered_cell = f"{bg_color} {cell:^3} \033[0m" if cell.strip() else f"{bg_color}     \033[0m"
            row_cells.append(centered_cell)

        print(f" {row} │" + "│".join(row_cells) + f"│ {row}")

        if row != 1:
            print("   ├─────┼─────┼─────┼─────┼─────┼─────┼─────┼─────┤")
        else:
            print("   └─────┴─────┴─────┴─────┴─────┴─────┴─────┴─────┘")

    print("     a     b     c     d     e     f     g     h  ")



# User input - place white figurine
while True:
    white_input = input("Choose bishop or pawn and place it on the board (e.g. 'pawn g3'): ").strip().lower()

    words = white_input.split()
    if len(words) == 2:  # Ensure correct format
        white_piece, white_pos = words
        if white_piece in white_figurines and white_pos in board_positions and board_positions[white_pos] == "":
            board_positions[white_pos] = white_figurines[white_piece]  # Store chess icon
            break
        else:
            print("Invalid figurine type or occupied square. Try again.")
    else:
        print("Invalid format! Please enter a figurine and position (e.g., 'bishop e5'). Try again.")

# User input - place black figurines
black_placements = {}
i = 0  # counter
first_entry = True  # Flag to track first input

while i < 16:  # Only valid placements counted toward the limit
    prompt_text = f"({i+1}/16) Enter black figurine and position (e.g. 'queen d8')"
    if not first_entry:
        prompt_text += " or type 'done' to finish"

    black_input = input(prompt_text + ": ").strip().lower()
    first_entry = False  # Update flag after first input

    if black_input == "done":
        if len(black_placements) == 0:
            print("Place at least one figurine before finishing.")
        else:
            break  # Stop early if the user is done

    words = black_input.split()
    if len(words) == 2:  # Ensure correct format
        black_piece, black_pos = words

        if black_piece in black_figurines:
            if can_place_black_figurine(black_piece):  # Check piece limit
                if black_pos in board_positions and board_positions[black_pos] == "":
                    board_positions[black_pos] = black_figurines[black_piece]["icon"]  # Store chess icon
                    black_placements[black_pos] = black_piece
                    black_placements_count[black_piece] += 1  # Track usage count
                    i += 1  # Only when a valid placement is made
                else:
                    print("Invalid position or occupied square. Try again.")
            else:
                print(f"Cannot place more {black_piece}s. Limit reached ({black_figurines[black_piece]['limit']}).")
        else:
            print("Invalid figurine type. Try again.")
    else:
        print("Invalid format! Enter figurine type and position (e.g. 'queen d8'). Try again.")


# Define white figurine attack schema
movement_schema = {
    "pawn": [(-1, 1), (1,1)],  # attacks diagonally only forward 1 square
    "bishop": [(1, 1), (-1, 1), (1, -1), (-1, -1)],  # Moves diagonally any distance
}

# Convert to x,y coordinates to calculate where a figurine can move
def chess_to_index(position):
    if len(position) != 2:
        return None
    col, row = position[0], position[1]
    if col not in "abcdefgh" or not row.isdigit():
        return None
    return ord(col) - ord("a"), int(row) - 1

def index_to_chess(index):
    col, row = chr(index[0] + ord("a")), index[1] + 1
    return f"{col}{row}"


def get_valid_moves(piece:str, current_pos):
    """Generate possible moves"""
    index = chess_to_index(current_pos)
    if index is None:
        print("Invalid position format.")
        return []

    x, y = index
    moves = []

    if piece == "bishop":
        for dx, dy in movement_schema["bishop"]:
            new_x, new_y = x + dx, y + dy
            while 0 <= new_x < 8 and 0 <= new_y < 8:
                move_pos = index_to_chess((new_x, new_y))
                moves.append(move_pos)

                # Break if black figurine is found
                if move_pos in black_placements:
                    break

                new_x += dx
                new_y += dy

    elif piece == "pawn":
        for dx, dy in movement_schema["pawn"]:
            new_x, new_y = x + dx, y + dy
            while 0 <= new_x < 8 and 0 <= new_y < 8:
                move_pos = index_to_chess((new_x, new_y))
                moves.append(move_pos)

                # Break if black figurine is found
                if move_pos in black_placements:
                    break

                new_x += dx
                new_y += dy

    elif piece in movement_schema:
        for dx, dy in movement_schema[piece]:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < 8 and 0 <= new_y < 8:
                moves.append(index_to_chess((new_x, new_y)))

    return moves


# Find the white piece and its position
for pos, symbol in board_positions.items():
    if symbol == white_figurines["bishop"] or symbol == white_figurines["pawn"]:
        white_piece = "bishop" if symbol == white_figurines["bishop"] else "pawn"
        white_pos = pos
        break


# Get possible moves for the white piece
white_moves = get_valid_moves(white_piece, white_pos)

# Find black pieces that can be captured
capturable_black = []
for move in white_moves:
    if move in black_placements:
        capturable_black.append((black_placements[move], move))

# Remove non-capturable black pieces from the board
for pos in black_placements:
    board_positions[pos] = "  "
for piece, pos in capturable_black:
    board_positions[pos] = black_figurines[piece]["icon"]

# Display the final board with only capturable pieces
print("\nThe black figurines that will be taken by your white figurine:", end=" ")

if capturable_black:
    print() # new line
    print_grid()
else:
    print("\033[1mNONE\033[0m")


